In [110]:
import os
import pandas as pd
import numpy as np

data_path = 'C:/Users/kshit/Desktop/ML_Assignment_2/'
movies_filename = 'movie.csv'
ratings_filename = 'rating.csv'
movie = pd.read_csv(
    os.path.join(data_path, movies_filename),

    usecols=['movieId', 'title'],
    dtype={'movieId': 'int32', 'title': 'str'})

rating = pd.read_csv(
    os.path.join(data_path, ratings_filename),
    usecols=['userId', 'movieId', 'rating', 'timestamp'],
    dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})

In [111]:
rating.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [112]:
movie.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [113]:
movie = movie.set_index('movieId')
movie['total_rating'] = rating.movieId.value_counts()
movie['mean_rating'] = rating.groupby('movieId').mean()['rating']
movie.head()

,title,total_rating,mean_rating
movieId,,,
1,Toy Story (1995),49695.0,3.921240
2,Jumanji (1995),22243.0,3.211977
3,Grumpier Old Men (1995),12735.0,3.151041
4,Waiting to Exhale (1995),2756.0,2.861393
5,Father of the Bride Part II (1995),12161.0,3.064592


In [125]:
movie.shape

(27278, 3)

In [126]:
rating=rating[:2000000]
rating.shape

(2000000, 4)

In [127]:
from scipy.sparse import csr_matrix
# pivot ratings into movie features
movie_features = rating.pivot(
    index='movieId',
    columns='userId',
    values='rating'
).fillna(0)

In [128]:
rating.shape[0]

2000000

In [129]:
mat_movie_features = csr_matrix(movie_features.values)

In [130]:
movie_features.head()

userId,1,2,3,4,5,6,7,8,9,10,...,13558,13559,13560,13561,13562,13563,13564,13565,13566,13567
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,4.0,0.0,0.0,5.0,0.0,4.0,0.0,4.0,...,3.0,4.0,0.0,2.0,4.0,0.0,5.0,4.5,0.0,0.0
2,3.5,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
3,0.0,4.0,0.0,0.0,0.0,3.0,3.0,5.0,0.0,0.0,...,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [149]:
# to calc the distance of two vectors
def distance(x, y):
    return np.linalg.norm(x - y)

# to get all ratings from an user
def user_ratings(user_id):
    user_r = rating[ rating.userId == user_id ]
    return user_r.drop(['userId', 'timestamp'], axis=1)

# to find the movies which both watched.
def merge(df1,df2):
    return df1.merge(df2, how='inner', on="movieId")

# to calc the distance of two users
def distance_between_users(user_id1, user_id2):
    vector_user1 = user_ratings(user_id1)
    vector_user2 = user_ratings(user_id2)
    vectors = merge(vector_user1, vector_user2)
    if vectors.empty:
        return (user_id1, user_id2, 100)
    return user_id1, user_id2, distance(vectors['rating_x'], vectors['rating_y'])

# Get all user ids
def get_users_ids():
    return list(rating.userId.unique())

# Using 10000 users for user id references
def center_from_all(user_id):
    if not user_id in rating.userId:
        return False   
    
    ids = get_users_ids()
    ids.remove(user_id)
    return [distance_between_users(user_id, for_id) for for_id in ids[:10000]]

In [151]:
# returning  movies information
def ids_to_movie(movieId):
    movie.loc[movies_recommend(1)].sort_values('mean_rating', ascending=False)
    
def movies_recommend(userId, k=10):
    points = center_from_all(userId)
    points = pd.DataFrame(points, columns=['myId','userId','distance'])
    points.sort_values('distance', axis=0, ascending=True, inplace=True)
    movies_list = list()

    my_ratings = ratings_by_user(userId)
    our_users = list(points.userId)

    for our_user in our_users:
        merged = my_ratings.merge(user_ratings(our_user), how='right', on="movieId")
        merged = merged[merged.rating_x.isna()]
        movies_list.append(merged)
        movies_list = pd.concat(movies_list)
        return movie.loc[movies_list.movieId].sort_values('mean_rating', ascending=False)[:k]

In [ ]:
import time
start = time.time()
movies_recommend(26)
end = time.time()
print("\n Time Taken",end-start)
movies_recommend(26)


 Time Taken 77.29464864730835


In [153]:
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import process
model_knn= NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20)

In [154]:
model_knn.fit(mat_movie_features)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=20)

In [155]:
def recommender(movie_name, data,model, n_recommendations ):
    model.fit(data)
    idx=process.extractOne(movie_name, movie['title'])[2]
    print('Movie Selected: ',movie['title'][idx], 'Index: ',idx)
    print('Searching for recommendations.....')
    distances, indices=model.kneighbors(data[idx], n_neighbors=n_recommendations)
    for i in indices:
        print(movie['title'][i].where(i!=idx))
    
recommender('fight club', mat_movie_features, model_knn,20)

Movie Selected:  Fight Club (1999) Index:  2959
Searching for recommendations.....
movieId
2959                                                  NaN
2522                                   Airport '77 (1977)
3738                        Sugarland Express, The (1974)
5204               Death Watch (La Mort en Direct) (1980)
6330                                 People I Know (2002)
4289    City of Women, The (Città delle donne, La) (1980)
3289         Not One Less (Yi ge dou bu neng shao) (1999)
5343                                     Temp, The (1993)
2761                               Iron Giant, The (1999)
4124                             Jaws: The Revenge (1987)
2783                           Tomb of Ligeia, The (1965)
5638       Godzilla vs. Mothra (Mosura tai Gojira) (1964)
3671                               Blazing Saddles (1974)
2757                                       Frances (1982)
3252                              Scent of a Woman (1992)
1831                                 Lo